## Generates Dummy Data For testing

In [1]:
%%writefile datagen_requirements.txt
pandas==2.2.1

Overwriting datagen_requirements.txt


In [2]:
!pip install -r datagen_requirements.txt

In [3]:
import csv
import random
import pandas as pd
from time import sleep
import datetime

In [4]:
tag200 = '2570/Ma_B.SLV.Counter.Counter0200'
tag202 = '2570/Ma_B.SLV.Counter.Counter0202'
product = 'WFI+9mg/g NACL 1x0.70ml PFS DSF4 BD'

In [5]:
print('coma seperated batches')
batches = input()

coma seperated batches


 ED1108


In [6]:
batches_to_generate = batches.split(', ')

In [7]:
print(batches_to_generate)

['ED1108']


In [8]:
configuration = []
stopper = []
number_of_stops = None
for batch in batches_to_generate:
    print('Details for batch %s' % batch)
    print('is_stopped')
    is_stopped = input()
    if is_stopped == '1':
        print('Numben of stops')
        number_of_stops = input()
    print('Start Date in dd-MM-yyyy')
    start_date = input()
    print('Start Date in HH:mm')
    start_time = input()
    print('Max Production Per Cycle')
    max_production_per_cycle = input()
    print('Total Production')
    total_production = input()
    print('Number of csv Generation Cycle')
    csv_gen_cycle = input()
    start_datetime = start_date + ' ' + start_time
    date_format = '%d-%m-%Y %H:%M'
    configuration.append({'Batch': batch, \
                          'Stopped': is_stopped == '1', \
                          'NumberOfStops': int(number_of_stops) if number_of_stops is not None else 0, \
                          'StartDateTime': datetime.datetime.strptime(start_datetime, date_format), \
                          'MaxCycleProduction': int(max_production_per_cycle), \
                          'TotalProduction': int(total_production), \
                          'CurrentTotalProduced': 0, \
                          'CurrentTotalError': 0, \
                          'CsvCycle': int(csv_gen_cycle)})
    stopper.append({'Batch': batch, 'IsStop': False})

Details for batch ED1108
is_stopped


 1


Numben of stops


 4


Start Date in dd-MM-yyyy


...

 185


Total Production


 1987564


Number of csv Generation Cycle


 15


In [9]:
print(configuration)

[{'Batch': 'ED1108', 'Stopped': True, 'NumberOfStops': 4, 'StartDateTime': datetime.datetime(2024, 6, 13, 18, 45), 'MaxCycleProduction': 185, 'TotalProduction': 1987564, 'CurrentTotalProduced': 0, 'CurrentTotalError': 0, 'CsvCycle': 15}]


In [10]:
def stopping_condition():
    control = True
    for stop_condition in stopper:
        control = control and stop_condition['IsStop']
    return control  

In [11]:
def generate_value(max_production, is_lower_prdouction_rate):
    range = max_production if not is_lower_prdouction_rate else int(40)
    generated_value = random.randrange(range)
    return generated_value if generated_value > 0 else 17

In [12]:
def write_to_csv(batch_name, file_id, dataframe):
    print(dataframe)
    file_name = 'Generated/' + batch_name + '_' + str(file_id) + '.csv'
    dataframe.to_csv(file_name, encoding='utf-8')

In [13]:
# add zeroes to current generated data
def add_zeroes(df, batch_name, start_time, sample_size):
    for i in range(sample_size):
        df.loc[len(df)] = [batch_name, product, tag200, start_time, 0]
        has_error_value = random.getrandbits(1)
        if has_error_value:
            df.loc[len(df)] = [batch_name, product, tag202, start_time, 0]
        start_time = start_time + datetime.timedelta(seconds=60)   

In [14]:
def same_time_stamp_different_value(df, conf, timestamp, is_lower_production_rate, current_production):
    gen1 = generate_value(conf['MaxCycleProduction'], is_lower_production_rate)
    gen2 =generate_value(conf['MaxCycleProduction'], is_lower_production_rate)
    df.loc[len(df)] = [conf['Batch'], product, tag200, timestamp, current_production + gen1]
    df.loc[len(df)] = [conf['Batch'], product, tag200, timestamp, current_production + gen2]
    if gen1 > gen2:
        conf['CurrentTotalProduced'] = conf['CurrentTotalProduced'] + gen1
        current_production += gen1
    else:
        conf['CurrentTotalProduced'] = conf['CurrentTotalProduced'] + gen2
        current_production += gen2
    return current_production

In [15]:
def same_value_different_timestamp(df, conf, timestamp, is_lower_production_rate, current_production):
    gen = generate_value(conf['MaxCycleProduction'], is_lower_production_rate)
    df.loc[len(df)] = [conf['Batch'], product, tag200, timestamp, current_production + gen]
    timestamp = timestamp + datetime.timedelta(seconds=60)
    df.loc[len(df)] = [conf['Batch'], product, tag200, timestamp, current_production+ gen]
    conf['CurrentTotalProduced'] = conf['CurrentTotalProduced'] + gen
    return current_production + gen

In [16]:
def generate_data_for_batch(conf):
    df = pd.DataFrame(columns = ['BATCH_ID', 'PRODUCT_ID', 'TAG_ID', 'TIMESTAMP', 'VALUE'])
    excel_file_no = 1
    current_size = 0
    current_error = 0
    number_of_stoppers = conf['NumberOfStops']
    stop_portion_size = int(conf['TotalProduction'] / number_of_stoppers)
    next_dt = conf['StartDateTime']
    # Add initial zeroes to batch 
    add_zeroes(df, conf['Batch'], next_dt, random.randrange(20))
    start_error = 0
    while conf['CurrentTotalProduced'] <= conf['TotalProduction']:
        if conf['CsvCycle'] > 1 and conf['TotalProduction'] / conf['CsvCycle'] * excel_file_no <= conf['CurrentTotalProduced']:
            print('Flush to CSV file')
            write_to_csv(conf['Batch'], excel_file_no, df)
            excel_file_no += 1
        # For first 2 production value do not generate error or get error in 50% of chance
        has_error_value = random.getrandbits(1) and start_error > 2
        # 60% chance has total production value
        has_total_value = random.randrange(5) % 2 == 0
        if start_error <= 2:
            start_error += 1
        # 10% of chance error value is greater than production
        is_error_value_more_than_total = random.randrange(10) % 9 == 0
        # first 10% of the production more prone to error and has lower production rate
        is_lower_production_rate = conf['TotalProduction'] / conf['TotalProduction'] * 100 < 10
        # 20% of the chance data has same timestamp for different values
        has_same_timestamp = random.randrange(20) % 19 == 0
        if has_same_timestamp:
            current_size = same_time_stamp_different_value(df, conf, next_dt, is_lower_production_rate, current_size)
        # 20% of chance same value with different timestamp
        has_same_value = random.randrange(20) % 19 == 0
        if  has_same_value:
            current_size = same_value_different_timestamp(df, conf, next_dt, is_lower_production_rate, current_size)
        if stop_portion_size < current_size:
            current_size = 0
            current_error = 0
            add_zeroes(df, conf['Batch'], next_dt, 15)
        if has_total_value:
            gen = generate_value(conf['MaxCycleProduction'], is_lower_production_rate)
            conf['CurrentTotalProduced'] = conf['CurrentTotalProduced'] + gen
            current_size += gen
            if gen > 0:
                df.loc[len(df) - 1] = [conf['Batch'], product, tag200, next_dt, current_size]
        if has_error_value:
            error = generate_value(conf['MaxCycleProduction'] if is_lower_production_rate else int(conf['MaxCycleProduction'] / 3), is_lower_production_rate)
            if error < gen and is_error_value_more_than_total:
                error = gen + 10
            conf['CurrentTotalError'] = conf['CurrentTotalError'] + error
            current_error += error
            if error > 0:
                df.loc[len(df) - 1] = [conf['Batch'], product, tag202, next_dt, current_error]
        next_dt = next_dt + datetime.timedelta(seconds=60)
    #Add Final Zeroes
    add_zeroes(df, conf['Batch'], next_dt, random.randrange(20))
    write_to_csv(conf['Batch'], excel_file_no, df)

In [17]:
for conf in configuration:
    print(conf)
    conf['CurrentTotalProduced'] = 0
    conf['CurrentTotalError'] = 0
    generate_data_for_batch(conf)

{'Batch': 'ED1108', 'Stopped': True, 'NumberOfStops': 4, 'StartDateTime': datetime.datetime(2024, 6, 13, 18, 45), 'MaxCycleProduction': 185, 'TotalProduction': 1987564, 'CurrentTotalProduced': 0, 'CurrentTotalError': 0, 'CsvCycle': 15}
Flush to CSV file
    BATCH_ID                           PRODUCT_ID  \
0     ED1108  WFI+9mg/g NACL 1x0.70ml PFS DSF4 BD   
1     ED1108  WFI+9mg/g NACL 1x0.70ml PFS DSF4 BD   
2     ED1108  WFI+9mg/g NACL 1x0.70ml PFS DSF4 BD   
3     ED1108  WFI+9mg/g NACL 1x0.70ml PFS DSF4 BD   
4     ED1108  WFI+9mg/g NACL 1x0.70ml PFS DSF4 BD   
..       ...                                  ...   
659   ED1108  WFI+9mg/g NACL 1x0.70ml PFS DSF4 BD   
660   ED1108  WFI+9mg/g NACL 1x0.70ml PFS DSF4 BD   
661   ED1108  WFI+9mg/g NACL 1x0.70ml PFS DSF4 BD   
662   ED1108  WFI+9mg/g NACL 1x0.70ml PFS DSF4 BD   
663   ED1108  WFI+9mg/g NACL 1x0.70ml PFS DSF4 BD   

                                TAG_ID           TIMESTAMP   VALUE  
0    2570/Ma_B.SLV.Counter.Counter0200 2